# TRAINING JUPYTER

### Parameter setting

In [1]:
# Imports et variables setting

# General Libs
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import os
import keras
import datetime
from contextlib import redirect_stdout

# Current files
from generator import Up_and_Down_Generator, FC_Generator
from preprocess import Level_Normalizer, DictPrepross

from utils import Load_FLX_dict, Plot_Batch
from CST import CST

data_folder=  CST.Data_folder(CST)
model_folder = CST.Model_folder(CST)
log_folder = CST.Log_folder(CST)

##### Dictionnary
np.random.seed(0)
D2 = []
D2 = [DictPrepross(['o3','pl'], [Level_Normalizer(False),Level_Normalizer(False)] )]
D = [Load_FLX_dict()]
##### Kernels

Klist = []
fit_genator = Up_and_Down_Generator(folder=data_folder, batch_size=512, train=True, preprocess_x=D)
D2[0]['pl'].fit(fit_genator[0][0][:,:,3])
D2[0]['o3'].fit(fit_genator[0][0][:,:,2])

##### Full Preprocessing :
FP = D + D2 #+ Klist
##### b_size
batch_size= 64

train_generator =  FC_Generator(folder=data_folder, batch_size=batch_size, train=True, preprocess_x=FP,custom_b_p_e = 50)
validation_generator =  FC_Generator(folder=data_folder, batch_size=5*batch_size, train=False, preprocess_x=FP, custom_b_p_e = 50)

header_x = train_generator.variables
header_y = train_generator.new_variables_pred

Using TensorFlow backend.
/usr/lib/python3/dist-packages/requests/__init__.py:80: RequestsDependencyWarning: urllib3 (1.24.2) or chardet (3.0.4) doesn't match a supported version!
  RequestsDependencyWarning)


## II) ARCHITECTURES

### BD or BD+Unet

In [2]:
from architectures import U_net, LossHistory, Generate_Log, Bidir_Causal_Conv
from architectures import total_loss, flxd_loss, flxu_loss, dfdts_loss

from keras import optimizers
import datetime

ups=5
pooling=22
units = [300, 100,72*3]

prefix = datetime.datetime.now().strftime("%Y%m%d%H%M%S")
archi = "FC"

#M_ffc = FFC_archi(ups=ups, pooling=pooling, BS=batch_size, list_of_unit=units )
M = FC_archi(ups, pooling, units, batch_size, reg=0.0001, o_channel=3, lev= 72, unique_var = 3, level_var=8)

LH = LossHistory()
rmsprop = optimizers.RMSprop(lr=0.01, rho=0.9, epsilon=None, decay=1.e-6)
M.compile(loss=total_loss_fc, optimizer=rmsprop, metrics=[flxd_loss_fc,flxu_loss_fc, dfdts_loss_fc])
M.summary()

ImportError: cannot import name 'U_net'

In [ ]:
def Bidir_Casual_Conv2(list_of_kernel_s, list_of_filters, ups, pooling, in_channel, o_channel, lev=CST.lev(CST)):
    """
    used as input for the Unet
    """
    Input0 = Input(shape=(lev, in_channel), name=Name('Input',0), dtype='float32')

    Flip_layer = lambda x: K.reverse(x, axes=0)
    I_cp = UpSampling1D(ups, name=Name('Up',0))(Input0)
    I_avg = AveragePooling1D(pooling, padding='same', stride=ups, name='AVG_p')(I_cp)
    I_avg_flip = Lambda(Flip_layer, name=Name('Flip',0))(I_avg)

    Conv1u = [I_avg]
    Conv1d = [I_avg_flip]
    ACT = []

    # Normal
    for i in range(len(list_of_filters[0])):
        Conv1u.append(Conv1D(filters = list_of_filters[0][i], kernel_size= list_of_kernel_s[0][i], \
                        padding='causal', activation='relu', name=Name("Conv_u",i+1), use_bias=True)(Conv1u[-1]))
    # Flipped
    for i in range(len(list_of_filters[1])):
        Conv1d.append(Conv1D(filters = list_of_filters[1][i], kernel_size= list_of_kernel_s[1][i],\
                        padding='causal', activation='relu', name=Name("Conv_d",i+1), use_bias=True)(Conv1d[-1]))

    C_flip = Lambda(Flip_layer,name=Name('Flip',1))(Conv1d[-1])
    C1d_prime = [Concatenate( name=Name('Concat',0))([Conv1u[-1], C_flip])]
    for i in range(len(list_of_filters[2])):
        if i==len(list_of_kernel_s)-1:
            C1d_prime.append(Conv1D(filters = list_of_filters[2][i], kernel_size=list_of_kernel_s[2][i], \
                            padding='causal', name=Name("Conv_conc",i), use_bias=False)(C1d_prime[-1]))
        else:
            C1d_prime.append(Conv1D(filters = list_of_filters[2][i], kernel_size=list_of_kernel_s[2][i], \
                            padding='causal', name=Name("Conv_conc",i), use_bias=False, activation='relu')(C1d_prime[-1]))
    return keras.Model(Input0, C1d_prime[-1])


def Unet2(list_of_kernels_s, list_of_filters, list_of_pooling, Div=3, lev=CST.lev(CST), in_channel=11, o_channel=CST.output_y(CST)):
    """
    Generate a Unet-Archictecture
    list_of_kernels : list of 2 lists containing the kernel size for convolution
    list_of_filters : list of 2 lists containing the number of filters for convolution
    Div : number of downscaling
    in_channel : number of inputs
    """
    Concats_l = []
    Upsamplings_l = []
    Convs_l1 = []
    Convs_l2 = []
    Poolings_l = []
# DownScaling
    Convs_l1.append(Input(name = 'Origin_Input',  dtype='float32', shape=(lev, in_channel)))
    for i in range(Div):
        Poolings_l.append(AveragePooling1D(list_of_kernels_s[0][i]-1, padding='same', stride=2, name=Name('AVG', i+1))(Convs_l1[-1]))
        Convs_l1.append(Conv1D(filters=list_of_filters[0][i], kernel_size=list_of_kernels_s[0][i], \
                                padding='same', activation='relu', name=Name('Conv1',i+1))( Poolings_l[-1] ))

# Operation done on the small dimension : here fc
    Convs_l2.append(Flatten()(Convs_l1[-1])  )
    Convs_l2.append(Dense( int(lev/2**Div) * list_of_filters[1][0]  )(Convs_l2[-1])  )
    Convs_l2.append(Reshape(name='Reshape',input_shape=Convs_l2[-1].shape ,\
                            target_shape=( int(lev/2**Div)  ,  list_of_filters[1][0] ))(Convs_l2[-1]))

# Upsampling and concats
    for i in range(Div):
        Concats_l.append(Concatenate( name=Name('Concat',i+1) )([Convs_l2[-1], Convs_l1[-i-1]]))
        Upsamplings_l.append(UpSampling1D(2, name=Name('Ups',i+1))(Concats_l[-1]))
        Convs_l2.append(Conv1D(filters=list_of_filters[2][i], kernel_size=list_of_kernels_s[2][i], \
                                padding='same', activation='relu', name=Name('Conv2',i+1))( Upsamplings_l[-1] ))
    Conv3 = Conv1D(filters=o_channel, kernel_size=1, padding='same', use_bias=False)(Convs_l2[-1])
    return keras.Model(Convs_l1[0],Conv3)
###### FC
        
    

In [3]:
M

NameError: name 'M' is not defined

In [ ]:
history = M_conv.fit_generator(generator=train_generator ,
                    validation_data=validation_generator,
                             shuffle=False,
                               callbacks = [LH],
                               epochs=50,
                               verbose=1)

Epoch 1/50
50/50 [==============================] - 4s 73ms/step - loss: 973479.4465 - flxd_loss: 141.3803 - flxu_loss: 49.9840 - dfdts_loss: 973288.1416 - val_loss: 392.6856 - val_flxd_loss: 107.3487 - val_flxu_loss: 29.8650 - val_dfdts_loss: 255.4719
Epoch 2/50
50/50 [==============================] - 2s 32ms/step - loss: 396.4329 - flxd_loss: 107.8757 - flxu_loss: 30.7457 - dfdts_loss: 257.8115 - val_loss: 406.8044 - val_flxd_loss: 110.6561 - val_flxu_loss: 31.5005 - val_dfdts_loss: 264.6477
Epoch 3/50
50/50 [==============================] - 2s 33ms/step - loss: 408.4122 - flxd_loss: 87.3486 - flxu_loss: 28.9185 - dfdts_loss: 292.1451 - val_loss: 443.2729 - val_flxd_loss: 95.7110 - val_flxu_loss: 29.3179 - val_dfdts_loss: 318.2440
Epoch 4/50
50/50 [==============================] - 2s 31ms/step - loss: 475.7501 - flxd_loss: 98.0895 - flxu_loss: 29.3522 - dfdts_loss: 348.3084 - val_loss: 444.6700 - val_flxd_loss: 95.7504 - val_flxu_loss: 30.7909 - val_dfdts_loss: 318.1287
Epoch 5/50

In [17]:
M_conv.save(  os.path.join(model_folder, prefix+archi+'.h5')  )
Generate_Log([M_ffc], history, LH, os.path.join(log_folder,prefix+archi+'.txt'), seed)

250

In [1]:
(1. / (1. + 1.e-6 * 68040))

0.9362945208044642

In [15]:
history.history

{'val_loss': [179.31939270019532],
 'val_flxd_loss': [84.54665435791016],
 'val_flxu_loss': [23.22427272796631],
 'val_dfdts_loss': [71.54846427917481],
 'loss': [124.8488102722168],
 'flxd_loss': [69.37535011291504],
 'flxu_loss': [21.22322248458862],
 'dfdts_loss': [34.25023921966553]}

### ANALYSIS :

In [ ]:
from contextlib import redirect_stdout
with open(os.path.join(log_folder, prefix), 'w') as f:
    with redirect_stdout(f):
        print('Seed {}'.format(seed))
        modelbd2.summary()
        modelbd.summary()
        for loss in FP:
            print(loss)
        print(history.history)

### VISUALISATION

In [2]:
# FCT PLOT
def Show_triple_diff(y,y0):
    j = 50 #np.random.randint(0)
    f = plt.figure( figsize=(15,8)   )
    ax = plt.subplot(131)
    lev= 35
    plt.plot(np.flip(y[:,:,0].T[:lev,j]) , np.arange(lev));
    plt.plot(np.flip(y0[:,:,0].T[:lev,j]) , np.arange(lev));
    ax.legend(["truth", "pred"])
    ax = plt.subplot(132)
    plt.plot(np.flip(y[:,:,1].T[:,j]) , np.arange(72));
    plt.plot(np.flip(y0[:,:,1].T[:,j]) , np.arange(72));
    ax.legend(["truth", "pred"])
    ax = plt.subplot(133)
    plt.plot(np.flip(y[:,:,2].T[:,j]) , np.arange(72));
    plt.plot(np.flip(y0[:,:,2].T[:,j]) , np.arange(72));
    ax.legend(["truth", "pred"])

def Difference(y,y0):
#    y_cumsum = np.cumsum(y[i])
#    y0_cumsum = np.cumsum(y0)
    return(  np.mean(np.square(y-y0)))

def Compare(y,y0, i=0):
    f=plt.figure( figsize=(15,8), dpi=80)
    ax= f.add_subplot(1,2,1)
    ax.plot(np.flip(y0[i]), np.arange(len(y0[i]))) 
    ax.plot(np.flip(y[i]), np.arange(len(y0[i]))) 
    ax.legend(["y pred", 'y truth'])
#    ax.title("Diff")
    y_cumsum = np.cumsum(y[i])
    y0_cumsum = np.cumsum(y0[i])
    ax= f.add_subplot(1,2,2)
    ax.plot(np.flip(y0_cumsum), np.arange(len(y0[i])))        
    ax.plot(np.flip(y_cumsum), np.arange(len(y0[i])))        
    ax.legend(["y pred", 'y truth'])
#    ax.title("Cumulative")
    plt.show()

def eliminate_var(m,x):
    O = []
    for i in range(11):
        x0= x.copy()
        x0*=0
        x0[:,:,i]=x[:,:,i]
        O.append(modelbd.predict(x0))
    return(O)

def Plot_Predictions(O, y, header):
    f=plt.figure( figsize=(15,10), dpi=80)
    for i,y0 in enumerate(O):
        ax= f.add_subplot(3,4,i+1)
        ax.set_title(header[i])
        for b in range(y0.shape[0]):
            ax.plot(np.flip(y0[b]), np.arange(len(y0[b])))
    ax= f.add_subplot(3,4,12)
    ax.set_title('flx')
    for b in range(y0.shape[0]):
        ax.plot(np.flip(y[b]), np.arange(len(y[b])))

def Normal2(x,header):
    O1 = []#['fcld', 'q','qi','ql','rl','ri']
    N = [ 'pl']
    STD = []
    STD2 = []
    for i, h in enumerate(header):
        if h in O1:
            x[:,:,i] = np.max(x[:,:,i], axis=1).reshape(x.shape[0],1)
        if h in N:
            #print(h, np.mean(x[:,:,i], axis=0)[32])
            x[:,:,i] -= np.mean(x[:,:,i], axis=0)
        if h in STD:
            x[:,:,i] /= (x[:,-1,i]+0.000000001).reshape(-1,1)         
        if h in STD2:
            x[:,:,i] /= (x[:,0,i]+0.000000001).reshape(-1,1)         
    return(x)


### SECOND ARCHITECTURE :
Not tested yet

# III) TRACKS 

### MODEL 2 :

- FCNN
- (with AE)

##### MODEL 2 : FCM-Final FC

##### MODEL 2 : U-net :
- use regular U-net so all layers affect each other and more stability

### MODEL 3 : Bidir-LSTM 
> Possible alternatives

- use two LSTM to show both impact of superior and inferior layer
- use attention model over it
- use w embeddings before

> TD

- Read git trez
- Read article of Hedge fun